In [51]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
import networkx as nx
import pandas as pd
from sklearn.linear_model import LinearRegression

In [79]:
# helper method for getting next step, returning the action to take, the supply in the next step, and the loan
def get_next_step(W,b_max,Q_max,B_init,forecast,gamma=0.999,alpha=1.0):

    S = forecast[:,:,0]
    D = forecast[:,:,1]
    N = len(S)
    F = len(S[0])
    G = np.diag(gamma**np.arange(F))
    
    B = cp.Variable([N,F+1])
    Q = cp.Variable([N,N,F])
    M = cp.Variable([N,F])
    
    obj_func = cp.sum([cp.norm(G@m_k,1) for m_k in M]) + alpha*cp.sum([cp.norm(cp.multiply(W,Q[:,:,t]),2) for t in range(F)])
    obj = cp.Minimize(obj_func)
    
    constr =  []
    
    # IC
    constr += [B[:,0] == B_init]
    # constr += [M>=0]
    
    
    # max storage
    for t in range(F+1):
        constr +=[B[:,t]<=b_max]
        
    # dynamics    
    for t in range(F):
        for k in range(N):
            constr += [B[k,t+1] == B[k,t] + S[k,t] - D[k,t] + cp.sum(Q[k,:,t])+ M[k,t]]
    
    # storage constraints
    for t in range(F+1):
        constr +=[0<= B[:,t]]
    
    
    ##### shipments
    for k in range(N):
        for l in range(N):
            constr +=[Q[k,l] == -Q[l,k]]
    
    for k in range(N):
        for t in range(F):
            constr += [cp.sum(Q[:,k,t]) <= B[k,t]]
    
    constr += [cp.norm(Q,'inf') <= Q_max]
    
    prob = cp.Problem(obj,constr)
    prob.solve()
    
    Q_opt = Q.value
    B_opt = B[:,1:].value
    M_opt = M.value

    return Q_opt[:,:,0],M_opt[:,0]

In [80]:
def forecast_loop(data,B_init,test_length,W,b_max,Q_max,forecaster,model,f=7,h=7,gamma=0.999,alpha=1.0,*args):
    data_real = data[:,-test_length:,:] # N x test_length x 2
    offset = len(data[0]) - test_length

    Q_mat = []
    B_mat = [B_init]
    M_mat = []
    
    for t in range(test_length - f):
        history = data[:,offset + t-h+1:offset + t+1,:]
        forecast = forecaster(model,history,*args) # N x f x 2
        Q_next,M_next = get_next_step(W,b_max,Q_max,B_mat[t],forecast,gamma,alpha)
        Q_mat.append(Q_next)
        M_mat.append(M_next)

        supply_real_offset = data_real[:, t+1, 0]
        demand_real_offset = data_real[:, t+1, 1]
        Q_next_sum = np.sum(Q_next, axis=1)
        
        B_next = B_mat[t][:] + supply_real_offset - demand_real_offset + Q_next_sum + M_next
        B_mat.append(B_next)

        print(B_next)

    return Q_mat,B_mat,M_mat

#### testing the loop on the AR forecaster

In [81]:
# importing data
experiment_type = "small"
data_directory = '../data/experiment_' + experiment_type + '/'
data = np.load(data_directory + 'network_params/data_network.npy')
data = np.transpose(data, (1,2, 0))

B_init = np.zeros(len(data))
f = 7
test_length = 365

G = nx.read_graphml(data_directory + "network_params/graph.graphml")
W = nx.to_numpy_array(G,weight='weight')

b_max = 2000*np.ones(len(data))
Q_max = 50
h=7
gamma=0.999
alpha=1.0

In [82]:
def data_slice(data, h, f):
    T = h + f
    samples = np.array([data[:,t:t + T,:] for t in range(len(data[0]) - T + 1)])
    return samples[:, :, :h, :], samples[:, : , h:, :]

def fit_AR_forecaster(hist_samples, future_samples):
  
    m, N, h, _ = hist_samples.shape
    _, _, f, _ = future_samples.shape
    
    hist_sample_matrix = hist_samples.reshape(m * N, h * 2)
    future_sample_matrix = future_samples.reshape(m * N, f * 2)
    
    model = LinearRegression()
    model.fit(hist_sample_matrix, future_sample_matrix)

    return model

def get_AR_forecast(AR_model, history):
    N, h, _ = history.shape
    f = len(AR_model.intercept_)//2
    history = history.reshape(N, 2 * h)

    forecast = AR_model.predict(history)
    return forecast.reshape(N,f,2)

In [83]:
data_train = data[:,:-3*test_length,:]
hist_train, future_train = data_slice(data_train, h = h, f = f)
AR_model = fit_AR_forecaster(hist_train, future_train)

In [84]:
Q,B,M = forecast_loop(
    data=data,
    B_init = B_init,
    test_length=test_length,
    W=W,
    b_max=b_max,
    Q_max=Q_max,
    forecaster=get_AR_forecast,
    model=AR_model,
    f=f,
    h=h,
    gamma=gamma,
    alpha=alpha
)

/usr/local/Caskroom/miniconda/base/envs/bloodflow/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:418: UserWarning: The problem has an expression with dimension greater than 2. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


[66.31526753 89.06088711 14.58766297 11.7863382 ]


/usr/local/Caskroom/miniconda/base/envs/bloodflow/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:418: UserWarning: The problem has an expression with dimension greater than 2. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


[140.44878833 188.38629289  31.23140448  24.65896952]


/usr/local/Caskroom/miniconda/base/envs/bloodflow/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:418: UserWarning: The problem has an expression with dimension greater than 2. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


...

/usr/local/Caskroom/miniconda/base/envs/bloodflow/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:418: UserWarning: The problem has an expression with dimension greater than 2. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(
/usr/local/Caskroom/miniconda/base/envs/bloodflow/lib/python3.12/site-packages/cvxpy/problems/problem.py:1481: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/usr/local/Caskroom/miniconda/base/envs/bloodflow/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:418: UserWarning: The problem has an expression with dimension greater than 2. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


[1999.45757287 2000.55897578  574.31414821  434.9987559 ]


/usr/local/Caskroom/miniconda/base/envs/bloodflow/lib/python3.12/site-packages/cvxpy/problems/problem.py:1481: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/usr/local/Caskroom/miniconda/base/envs/bloodflow/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:418: UserWarning: The problem has an expression with dimension greater than 2. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


[2008.50019972 2011.63759646  596.51098811  450.39323671]


TypeError: 'NoneType' object is not subscriptable

In [ ]:
print(Q)